# Learning Neo4j and Py2neo

### To start and stop Neo4j server

Because there was an error showing at the start of server saying pid was not created ( and a bunch of othet issues which might have been because of this) this was done, manually, Which seems to have become a work around for the issue

` cd /var/run`

` sudo mkdir neo4j`


To start the server

` sudo neo4j start`


To access the neo4j-browser `http://localhost:7474/browser`

default user : neo4j

default password: neo4j, new password I set: password

To stop the server

` sudo neo4j stop`


## Basic Design of a bible Graph

![title](Greek_linked_to_TW_Strongs2.png)


## Useful Cypher queries

To match a path from bible node to words of a verse

```
MATCH p=((bib:BIBLE)<-[:BELONGS_TO]-(b:BOOK)<-[:BELONGS_TO]-(c:CHAPTER)<-[:BELONGS_TO]-(v:VERSE)<-[:BELONGS_TO]-(w:WORD))
WHERE bib.name='Hindi_IRV4' and b.number='40' and c.number='5' and v.number='3' 
RETURN p
```

To retrieve just the text, from the above verse

```
MATCH p=((bib:BIBLE)<-[:BELONGS_TO]-(b:BOOK)<-[:BELONGS_TO]-(c:CHAPTER)<-[:BELONGS_TO]-(v:VERSE)<-[:BELONGS_TO]-(w:WORD))
WHERE bib.name='Hindi_IRV4' and b.number='40' and c.number='5' and v.number='3' 
RETURN w.literalform ORDER BY w.position
```


To view UGNT bible with links to Translation words and Strong Numbers

```
MATCH p=((bib:BIBLE)<-[:BELONGS_TO]-(:BOOK)<-[:BELONGS_TO]-(:CHAPTER)<-[:BELONGS_TO]-(:VERSE)<-[:BELONGS_TO]-(w:WORD)-[:IS_TW]->(:TRANSLATION_WORD)-[:BELONGS_TO]->(:DICTIONARY))
WITH p,w
MATCH (w)-[:HAS_LEMMA]->(n:STRONG_NUM)-[:BELONGS_TO]->(d:DICTIONARY)
RETURN p,n,d limit 5
```

To find the occurences of a word/strongs in a bible

```
MATCH p=((bib:BIBLE)<-[:BELONGS_TO]-(b:BOOK)<-[:BELONGS_TO]-(c:CHAPTER)<-[:BELONGS_TO]-(v:VERSE)<-[:BELONGS_TO]-(w:WORD)-[:HAS_LEMMA]->(s:STRONG_NUM))
WHERE s.id = 2424 and bib.name='UGNT4'
RETURN p LIMIT 5
```

To see how a verse is in various bibles

```
MATCH p=((bib:BIBLE)<-[:BELONGS_TO]-(b:BOOK)<-[:BELONGS_TO]-(c:CHAPTER)<-[:BELONGS_TO]-(v:VERSE)<-[:BELONGS_TO]-(w:WORD))
WHERE b.number='40' and c.number='5' and v.number='3' 
RETURN p
```

To find all the translations of a particular word/lemma

```
MATCH (w:WORD)-[:HAS_LEMMA]->(s:STRONG_NUM),
(a:WORD)-[:ALIGNS_TO]-(w)
WHERE s.id = 2424
RETURN w, a limit 3
```

To find the names from the Bibles without names marked

```
match (w:WORD)-[:ALIGNS_TO]-(:WORD)-[:IS_TW]-(tw:TRANSLATION_WORD) where tw.type='names' with distinct tw , w return w limit 10
```

## Neo4j DB Admin

### Export

To export the graph DB, stop the neo4j service/server, and then,

`sudo neo4j-admin dump --database=bible-linkua --to=<destination-path>`

### Import

To load an archived Graph into the DB, stop the neo4j service/server, and then,

`sudo neo4j-admin load --from=Documents/Graph_DB/Neo4j/graph__testAlignedFeb4.db --database=bible-linkua`


### Delete

To remove a database from the neo4j, go to `/var/lib/neo4j/data/databases` and delete the folder in the corresponding name 

### Change DB

To make neo4j load a different database, go to `/etc/neo4j/neo4j.conf` and update the line, `dbms.active_database=bible-linkua`


## Py2neo

In [1]:
import py2neo
from py2neo import Graph, NodeMatcher
from py2neo.data import Node, Relationship

graph = Graph(password="password")

`Node()` creates a new node object in the python run time, but not in the actual graph DB. For that we have to use `create` or `merge`.

Merging a node, creates a new node if there is no node already present in the graph, with the specified label, primary key, and value. If one such node is present, that would be matched. 

In [ ]:
bibleNode = Node("BIBLE", name=biblename)
bibleNode['version'] = 4.0
bibleNode['expandedName'] = "Unfoldingword Greek New Testament"

tx = graph.begin()
tx.merge(bibleNode,'BIBLE','name')
tx.commit()

We can match a relationship using `graph.match()`. But its better to have at least one bound node while doing so. Here `bibleNode` is the bound node and other is specified as `None`.

`Relationship()` functions similar to `Node()`

In [ ]:
BookName = 'Matthew'
BookNum = '40'

tx = graph.begin()

bookNode = Node("BOOK",name=BookName,number=BookNum)
found = False
for rel in graph.match((None,bibleNode), r_type="BELONGS_TO"):
    if rel.start_node["name"]==BookName:
        bookNode = rel.start_node
        found =True 
if not found:
    bookRel = Relationship(bookNode,"BELONGS_TO",bibleNode)
    bookRel['atIndex'] = BookNum
    tx.create(bookRel)
    print("creating new book-->bible relation")
tx.commit()